# High Performance Jupyter

## Good ol' PyData

|<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Pandas_logo.svg/1200px-Pandas_logo.svg.png" width="200" /> | <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/31/NumPy_logo_2020.svg/2880px-NumPy_logo_2020.svg.png" width="200" /> | <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Scikit_learn_logo_small.svg/1280px-Scikit_learn_logo_small.svg.png" width="200" />|
| -- | -- | -- |

Let's do our usual analysis on a laptop-sized machine with a dataset that fits comfortably in memory. This notebook should execute on any machine with >4GB RAM.

Outputs here are from a 2019 Macbook Pro (6 cores, 32GB RAM)

In [1]:
import pandas as pd
import numpy as np
import datetime
import s3fs
import warnings
warnings.simplefilter("ignore")

data_path = 's3://nyc-tlc/trip data'
seed = 42

# Load and explore data

We are using the [NYC Taxi data](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page), which contains several publicly-available datasets about taxi and rideshare rides taken in New York City.

Data is available from 2009 to 2020, but for this exercise we will use 2019 data only. Take care when using other data, as the schemas in the CSV files changed over the years. Most notably, in mid-2016 latitude and longitude fields were replaced with more generic taxi zones for privacy reasons.

Files are hosted in this S3 location: `s3://nyc-tlc/trip data`. The dataframe libraries we are using (`pandas`, `dask.dataframe`, `cudf`) all support reading directly from S3 so we don't have to download any files to our local filesystem.

We can use the `s3fs` package to explore the files that are on S3. We are dealing with a public bucket so there is no need to authenticate (`anon=True`).

In [2]:
fs = s3fs.S3FileSystem(anon=True)

files = fs.glob('s3://nyc-tlc/trip data/yellow_tripdata_*')
len(files), files[:5], files[-5:]

(138,
 ['nyc-tlc/trip data/yellow_tripdata_2009-01.csv',
  'nyc-tlc/trip data/yellow_tripdata_2009-02.csv',
  'nyc-tlc/trip data/yellow_tripdata_2009-03.csv',
  'nyc-tlc/trip data/yellow_tripdata_2009-04.csv',
  'nyc-tlc/trip data/yellow_tripdata_2009-05.csv'],
 ['nyc-tlc/trip data/yellow_tripdata_2020-02.csv',
  'nyc-tlc/trip data/yellow_tripdata_2020-03.csv',
  'nyc-tlc/trip data/yellow_tripdata_2020-04.csv',
  'nyc-tlc/trip data/yellow_tripdata_2020-05.csv',
  'nyc-tlc/trip data/yellow_tripdata_2020-06.csv'])

<br>
It looks like there's one file per month. Let's see how big the files are for 2019.

In [3]:
files_2019 = fs.glob('s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv')
file_sizes_2019 = [fs.du(f) for f in files_2019] 

print(f'2019 avg size (MB): {np.round(np.mean(file_sizes_2019) / 1e6)}')
print(f'2019 total size (GB): {np.round(np.sum(file_sizes_2019) / 1e9)}')

2019 avg size (MB): 650.0
2019 total size (GB): 8.0


<br>The following cell pulls the file down from S3 and loads the full contents into a dataframe - it'll take a couple minutes.

In [4]:
%%time

taxi = pd.read_csv(
        fs.open(f'{data_path}/yellow_tripdata_2019-01.csv'),
        parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'],
)

CPU times: user 22.4 s, sys: 2.23 s, total: 24.6 s
Wall time: 1min 20s


In [5]:
print(f"Row count: {len(taxi)}")
print(f"Size in GB: {taxi.memory_usage(deep=True).sum() / 1e9}")

Row count: 7667792
Size in GB: 1.487551776


In [6]:
taxi.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN


In [7]:
taxi.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
RatecodeID                        int64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
dtype: object

In [8]:
%%time 
np.round(taxi.describe(), 3).T

CPU times: user 3.73 s, sys: 590 ms, total: 4.32 s
Wall time: 4.33 s


,count,mean,std,min,25%,50%,75%,max
VendorID,7667792.0,1.637,0.540,1.0,1.00,2.00,2.00,4.00
passenger_count,7667792.0,1.567,1.224,0.0,1.00,1.00,2.00,9.00
trip_distance,7667792.0,2.801,3.738,0.0,0.90,1.53,2.80,831.80
RatecodeID,7667792.0,1.058,0.678,1.0,1.00,1.00,1.00,99.00
PULocationID,7667792.0,165.501,66.392,1.0,130.00,162.00,234.00,265.00
DOLocationID,7667792.0,163.753,70.364,1.0,113.00,162.00,234.00,265.00
payment_type,7667792.0,1.292,0.473,1.0,1.00,1.00,2.00,4.00
fare_amount,7667792.0,12.409,262.072,-362.0,6.00,8.50,13.50,623259.86
extra,7667792.0,0.328,0.507,-60.0,0.00,0.00,0.50,535.38
mta_tax,7667792.0,0.497,0.053,-0.5,0.50,0.50,0.50,60.80


# Feature engineering

We're going to predict "high-tip" rides, meaning rides where the tip percentage is greater than 20%. Tip is defined as the `tip_amount / fare_amount`.

In [9]:
numeric_feat = [
    'pickup_weekday', 
    'pickup_hour', 
    'pickup_week_hour', 
    'pickup_minute', 
    'passenger_count',
]
categorical_feat = [
    'PULocationID', 
    'DOLocationID',
]
features = numeric_feat + categorical_feat
y_col = 'high_tip'

In [10]:
def prep_df(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Generate features from a raw taxi dataframe.
    '''
    df = df[df.fare_amount > 0]  # avoid divide-by-zero
    df['tip_fraction'] = df.tip_amount / df.fare_amount
    df['high_tip'] = (df['tip_fraction'] > 0.2) # class label
    
    df['pickup_weekday'] = df.tpep_pickup_datetime.dt.weekday
    df['pickup_weekofyear'] = df.tpep_pickup_datetime.dt.weekofyear
    df['pickup_hour'] = df.tpep_pickup_datetime.dt.hour
    df['pickup_week_hour'] = (df.pickup_weekday * 24) + df.pickup_hour
    df['pickup_minute'] = df.tpep_pickup_datetime.dt.minute
    df = df[features + [y_col]].astype(float).fillna(-1)
    
    return df
    
taxi = prep_df(taxi)

In [11]:
taxi.head()

,pickup_weekday,pickup_hour,pickup_week_hour,pickup_minute,passenger_count,PULocationID,DOLocationID,high_tip
0,1.0,0.0,24.0,46.0,1.0,151.0,239.0,1.0
1,1.0,0.0,24.0,59.0,1.0,239.0,246.0,0.0
2,4.0,13.0,109.0,48.0,3.0,236.0,236.0,0.0
3,2.0,15.0,63.0,52.0,5.0,193.0,193.0,0.0
4,2.0,15.0,63.0,56.0,5.0,193.0,193.0,0.0


# Machine learning

We'll cover two different machine learning use cases that have differing resource constraints. First, a compute-bound problem: hyperparameter tuning with a linear model on a small dataset. Then, a compute- _and_ memory-bound problem: random forest on a large dataset. 

## Hyperparameter tuning 

Grid search with a logistic regression model. We'll sample down the data to ensure this is a compute-bound problem. 



In [12]:
taxi_sample = taxi.sample(frac=0.05, replace=False, random_state=seed)
taxi_sample.shape

(382912, 8)

Set up the pipeline and grid search. Setting `n_jobs=-1` tells scikit-learn to use all available cores on this machine to train models.

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV

lr = LogisticRegression(
    solver='saga',
    penalty='elasticnet', 
    l1_ratio=0.5,
    max_iter=100, 
    random_state=seed,
)

pipeline = Pipeline(steps=[
    ('preprocess', ColumnTransformer(transformers=[
        ('num', StandardScaler(), numeric_feat),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse=False), categorical_feat),
    ])),
    ('clf', lr),
])

params = {
    'clf__l1_ratio': [0.2, 0.3, 0.5, 0.7, 0.9],
}

grid_search = GridSearchCV(
    pipeline, 
    params, 
    cv=3, 
    n_jobs=-1, 
    verbose=1,
    scoring='accuracy',
)

In [14]:
%%time
_ = pipeline.fit(taxi_sample[features], taxi_sample[y_col])

CPU times: user 3min 44s, sys: 1.11 s, total: 3min 45s
Wall time: 3min 46s


In [15]:
%%time
_ = grid_search.fit(taxi_sample[features], taxi_sample[y_col])
grid_search.best_score_

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  15 | elapsed:  5.0min remaining:  4.4min
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  7.7min finished


CPU times: user 3min 35s, sys: 1.44 s, total: 3min 36s
Wall time: 11min 17s


0.5612882291887004

## Random forest

We're only training one model in this case, so we'll do a train/test split from the full data.

In [16]:
%%time
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    taxi[features], taxi[y_col], test_size=0.33, random_state=seed)

CPU times: user 2.09 s, sys: 687 ms, total: 2.77 s
Wall time: 2.77 s


Setting `n_jobs=-1` tells scikit-learn to use all available cores on this machine to train the forest.

In [17]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(
    n_estimators=100, 
    max_depth=5, 
    random_state=seed,
    n_jobs=-1,
)

In [18]:
%%time
_ = rfc.fit(X_train, y_train)

CPU times: user 24min 31s, sys: 18.5 s, total: 24min 50s
Wall time: 2min 29s


In [19]:
%%time
from sklearn.metrics import roc_auc_score

# get test metrics
preds = rfc.predict_proba(X_test)[:, 1]
roc_auc_score(y_test, preds)

CPU times: user 22.5 s, sys: 663 ms, total: 23.2 s
Wall time: 3.16 s


0.5501763462272633

<br>
<br>

# Moar data!!

That wasn't so bad right?? Time for more data!

![](https://i.chzbgr.com/full/6993318656/hC83012C2/analyze-all-the-data)

In [20]:
files_2019

['nyc-tlc/trip data/yellow_tripdata_2019-01.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-02.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-03.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-04.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-05.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-06.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-07.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-08.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-09.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-10.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-11.csv',
 'nyc-tlc/trip data/yellow_tripdata_2019-12.csv']

In [21]:
%%time

dfs = []
for f in files_2019:
    df = pd.read_csv(
        fs.open(f's3://{f}'),
        parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'],
    )
    dfs.append(df)
taxi_2019 = pd.concat(dfs)

CPU times: user 4min 25s, sys: 36.9 s, total: 5min 2s
Wall time: 16min 1s


And now we wait...

![](https://i.kym-cdn.com/entries/icons/original/000/010/437/Oneeternitylater.jpg)

It may eventually complete if your laptop has enough swap space. Worst case, you will run out of memory and your Jupyter kernel will die.

Which might leave you feeling a little like...

![](https://memegenerator.net/img/instances/61402104.jpg)

# Do not fear!

We're just getting started! Check out [dask.ipynb](dask.ipynb) to see how we can analyze all the files even if they don't fit in RAM.